In [1]:
# imports
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd

In [2]:
# downloading stopwords should they not be present
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jasonschenck/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:

raw_bible = open('bibledata.dat', 'r').read()

In [4]:
# In XML, the raw text content is arranged and separated by tagging. Below I will extract only data from text tags
# First, instantiate beautiful 'soup' with params (data_file,'file type') Note var name must be soup!
soup = BeautifulSoup(raw_bible, 'lxml')


In [5]:
# Filter the soup for only the values found between the <text> tags, rename the variable for ease of reading.
raw1 = soup.findAll('v')
#print(raw1)

# Generate the corpus with some list comprehension, iterates for all documents all text and appends docs to postDocs.
# Also, now the data will no longer be just text, so new var name postDocs 
verses = [x.text for x in raw1]


In [6]:
len(verses)

31063

In [7]:
verses = [x.lower() for x in verses]

In [8]:
len(verses)

31063

In [9]:
len(verses)

31063

In [ ]:
verses2 = list()
i = 0
while i < len(verses):
    verses2.append(verses[i] + verses[i + 1] + verses[i + 2] + verses[i + 3] + verses[i + 5])
    if (i > 0):
        i = i + 5

In [ ]:
len(verses2)

In [ ]:
verses3 = list()
i = 0
while i < len(verses2):
    verses3.append(verses2[i] + verses2[i + 1])
    i = i + 2
    

In [ ]:
len(verses3)

In [ ]:
verses4 = list()
i = 0
while i < len(verses3):
    verses4.append(verses3[i] + verses3[i + 1] + verses3[i + 2])
    i = i + 3


In [ ]:
len(verses4)

In [ ]:
verses5 = list()
i = 0
while i < len(verses4):
    verses5.append(verses4[i] + verses4[i + 1] + verses4[i + 2])
    i = i + 3

In [10]:
stopset = set(stopwords.words('english'))
stopset.update(['&#x27;s','x27','/',',','.','#','lt','p','/p','br','amp','quot','field','font','normal','span','0px','rgb','style','51', 
                'spacing','text','helvetica','size','family', 'space', 'arial', 'height', 'indent', 'letter'
                'line','none','sans','serif','transform','line','variant','weight','times', 'new','strong', 'video',
                'title','white','word','letter', 'roman','0pt','16','color','12','14','21', 'neue', 'apple', 'class',  ])


In [11]:
# Import the Terrier stopset from file, union with existing stopset
terrierstopset = open('terrierstopset.txt', 'r').read()
stopset = set(stopset).union(set(terrierstopset))

In [12]:
# Before...
print(len(verses4))

NameError: name 'verses4' is not defined

In [13]:
# Define the vectorizer model
vectorizer = TfidfVectorizer(stop_words=stopset, use_idf=True, ngram_range=(4, 6))

# Fit the corpus data
X = vectorizer.fit_transform(verses)

In [14]:
# Tada! This is now the output of the first document in the corpus, in sparse IDF matrix form.
print(X[0])

  (0, 151890)	0.584674333725
  (0, 198290)	0.562416079933
  (0, 151891)	0.584674333725


In [15]:
# The current shape is (documents, terms)
X.shape

(31063, 625970)

In [16]:
# Begin by defining the TruncatedSVD model (num rows/docs?, how many passes over the data (epochs)? )
#Note: n_iter defaults to 5 if not passed, and 1 if using partial_fit
lsa = TruncatedSVD(n_components=100, n_iter=10)

# Fit the model
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=100, n_iter=10,
       random_state=None, tol=0.0)

In [17]:
import sys
print (sys.version)

3.5.2 |Anaconda 4.2.0 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]


In [18]:
terms = vectorizer.get_feature_names()
for i, comp in enumerate(lsa.components_): 

    termsInComp = zip (terms,comp)
    sortedTerms =  sorted(termsInComp, key=lambda x: x[1], reverse=True) [:10]
    print("Concept %d:" % i )
    for term in sortedTerms:
        print(term[0])
    print (" ")

Concept 0:
male sheep one lamb
male sheep one lamb first
male sheep one lamb first year
one lamb first year burned
one lamb first year burned offering
one male sheep one
one male sheep one lamb
one male sheep one lamb first
one young ox one
one young ox one male
 
Concept 1:
male goats sin offering
one male goats sin
one male goats sin offering
came jerusalem three days
mouths voice eyes see
havilah jobab sons joktan
ophir havilah jobab sons
ophir havilah jobab sons joktan
children harim thousand seventeen
lord god judge earth
 
Concept 2:
basin seventy shekels scale
basin seventy shekels scale holy
basin seventy shekels scale holy place
full best meal mixed
full best meal mixed oil
full best meal mixed oil meal
holy place two full
holy place two full best
holy place two full best meal
hundred thirty shekels one
 
Concept 3:
gold spoon ten shekels
gold spoon ten shekels full
gold spoon ten shekels full spice
one gold spoon ten
one gold spoon ten shekels
one gold spoon ten shekels full
